#### Architecture - Semantic Search using Elastic Search


* Two very important concepts in Elasticsearch are documents and indexes.

* A document is collection of fields with their associated values. 

* To work with Elasticsearch you have to organize your data into documents, and then add all your documents to an index. 

* Index as a collection of documents that is stored in a highly optimized format designed to perform efficient searches.

#### Step 1: Prepare documents

In [27]:
import json

with open('../01-intro/documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [28]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[66]

{'text': 'When you run this command second time\ndocker run -it \\\n-e POSTGRES_USER="root" \\\n-e POSTGRES_PASSWORD="root" \\\n-e POSTGRES_DB="ny_taxi" \\\n-v <your path>:/var/lib/postgresql/data \\\n-p 5432:5432 \\\npostgres:13\nThe error message above could happen. That means you should not mount on the second run. This command helped me:\nWhen you run this command second time\ndocker run -it \\\n-e POSTGRES_USER="root" \\\n-e POSTGRES_PASSWORD="root" \\\n-e POSTGRES_DB="ny_taxi" \\\n-p 5432:5432 \\\npostgres:13',
 'section': 'Module 1: Docker and Terraform',
 'question': "Docker - Error response from daemon: error while creating buildmount source path '/run/desktop/mnt/host/c/<your path>': mkdir /run/desktop/mnt/host/c: file exists",
 'course': 'data-engineering-zoomcamp'}

#### Step 2: Create Embeddings using Pretrained Models

Sentence Transformers documentation here: https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

In [29]:
# This is a new library compared to the previous modules. 
# Please perform "pip install sentence_transformers==2.7.0"
from sentence_transformers import SentenceTransformer

# if you get an error do the following:
# 1. Uninstall numpy 
# 2. Uninstall torch
# 3. pip install numpy==1.26.4
# 4. pip install torch
# run the above cell, it should work
model = SentenceTransformer("all-mpnet-base-v2")

In [30]:
len(model.encode("This is a simple sentence"))

768

In [31]:
model.encode("vlad")

array([ 2.75906026e-02, -5.84350042e-02, -1.00285737e-02, -3.14118937e-02,
       -2.39794049e-02, -2.29688659e-02, -2.19572205e-02,  1.32297454e-02,
        2.62586940e-02,  1.20627787e-02,  7.82828331e-02, -5.28353192e-02,
       -5.11657558e-02, -5.18437512e-02,  4.85680737e-02, -3.57914828e-02,
        3.33855227e-02, -2.40198057e-02, -6.30485639e-02, -2.15416383e-02,
       -1.39756482e-02, -4.12000483e-03, -8.50767922e-03,  1.04645290e-03,
        7.02049062e-02, -1.19504528e-02, -2.05793092e-03,  7.96469674e-03,
        8.55446607e-03, -5.36464201e-03,  4.78219986e-03,  7.92132597e-03,
       -8.65927548e-04,  3.56353782e-02,  2.22807057e-06, -2.42679529e-02,
        2.68783458e-02, -1.51981227e-02,  7.34484522e-03,  1.41664036e-02,
        8.32456052e-02, -4.15550470e-02, -3.17887515e-02, -1.64508969e-02,
       -2.03465261e-02,  2.85946764e-02,  5.84209375e-02, -1.41677642e-02,
       -2.16378178e-02,  1.08076110e-02,  1.63377058e-02, -1.64269097e-02,
       -6.96457475e-02, -

In [32]:
documents[1], documents[666]

({'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'TODO',
  'section': '8. Neural Networks and Deep Learning',
  'question': 'How to get started with Week 8?',
  'course': 'machine-learning-zoomcamp'})

In [33]:
#created the dense vector using the pre-trained model
operations = []
for doc in documents:
    # Transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

In [34]:
operations[42]

{'text': 'Error: Makefile:2: *** missing separator.  Stop.\nSolution: Tabs in document should be converted to Tab instead of spaces. Follow this stack.',
 'section': 'General course-related questions',
 'question': 'VS Code: Tab using spaces',
 'course': 'data-engineering-zoomcamp',
 'text_vector': [0.011162682436406612,
  -0.01395924761891365,
  -0.022253552451729774,
  -0.05261039361357689,
  0.07844026386737823,
  0.010303561575710773,
  0.00781871285289526,
  0.049858588725328445,
  0.0033145220950245857,
  -0.005014972295612097,
  0.03264535591006279,
  0.05258861556649208,
  0.015281593427062035,
  0.059199556708335876,
  -0.021445900201797485,
  0.05511583760380745,
  0.04077698290348053,
  0.0377715639770031,
  -0.0006105588981881738,
  0.06405871361494064,
  -0.009927475824952126,
  0.06993427127599716,
  -0.006780145689845085,
  -0.01300816796720028,
  -0.021133417263627052,
  -0.058636751025915146,
  -0.00869174487888813,
  -0.02914070524275303,
  -0.00030846832669340074,
  

#### Step 3: Setup ElasticSearch connection 

In [35]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '3cbc31fe41ed', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'sfyC9chHRoSSF3OqTT6f1g', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

#### Step 4: Create Mappings and Index

* Mapping is the process of defining how a document, and the fields it contains, are stored and indexed.

* Each document is a collection of fields, which each have their own data type.

* We can compare mapping to a database schema in how it describes the fields and properties that documents hold, the datatype of each field (e.g., string, integer, or date), and how those fields should be indexed and stored 



In [36]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [37]:
index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

#### Step 5: Add documents into index

In [38]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

#### Step 6: Create end user query

In [39]:
search_term = "windows or mac?"
vector_search_term = model.encode(search_term)

In [40]:
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000, 
}

In [41]:
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'Fi_psZABeltwWQE9Fmi1',
  '_score': 0.71479183,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-questions',
  '_id': 'KS_psZABeltwWQE9K2tl',
  '_score': 0.6134736,
  '_source': {'question': 'WSL instructions',
   'course': 'mlops-zoomcamp',
   'section': 'Module 1: Introduction',
   'text': 'If you wish to use WSL on your windows machine, here are the setup instructions:\nCommand: Sudo apt install wget\nGet Anaconda download address here. wget <download address>\nTurn on Docker Desktop WFree Download | AnacondaSL2\nCommand: git clone <github repository address>\nVSCODE on WSL\nJupyter: pip3 install jupyter\nAdded by Gregory Morris (gwm1980@gmail.com)\nAll in all softwares a

#### Step 7: Perform Keyword search with Semantic Search (Hybrid/Advanced Search)

In [42]:
# Note: I made a minor modification to the query shown in the notebook here
# (compare to the one shown in the video)
# Included "knn" in the search query (to perform a semantic search) along with the filter  
knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000
}

In [43]:
response = es_client.search(
    index=index_name,
    query={
        "match": {"section": "General course-related questions"},
    },
    knn=knn_query,
    size=5
)

In [44]:
response["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'Fi_psZABeltwWQE9Fmi1',
  '_score': 11.466713,
  '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
   'section': 'General course-related questions',
   'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'text_vector': [-0.026965443044900894,
    -0.0006260006339289248,
    -0.016629481688141823,
    0.05285147204995155,
    0.05476532131433487,
    -0.031339917331933975,
    0.029942616820335388,
    -0.048085641115903854,
    0.04467550665140152,
    0.005839488934725523,
    0.016233086585998535,
    0.012001189403235912,
    -0.031222380697727203,
    0.016600608825683594,
    -0.04886902496218681,
    -0.06496305018663406,
    0.04643420875072479,
    -0.009297692216932774,
    -0.06425277888774872,
    -0.013732724823057652,
    -0.015976158902049065,
    0.00862953532487154,
    -0.0244789458

In [45]:
knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000
}

response = es_client.search(
    index=index_name,
    query={
        "match": {"course": "data-engineering-zoomcamp"},
    },
    knn=knn_query,
    size=5
)

In [46]:
response["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'Fi_psZABeltwWQE9Fmi1',
  '_score': 1.4462318,
  '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
   'section': 'General course-related questions',
   'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'text_vector': [-0.026965443044900894,
    -0.0006260006339289248,
    -0.016629481688141823,
    0.05285147204995155,
    0.05476532131433487,
    -0.031339917331933975,
    0.029942616820335388,
    -0.048085641115903854,
    0.04467550665140152,
    0.005839488934725523,
    0.016233086585998535,
    0.012001189403235912,
    -0.031222380697727203,
    0.016600608825683594,
    -0.04886902496218681,
    -0.06496305018663406,
    0.04643420875072479,
    -0.009297692216932774,
    -0.06425277888774872,
    -0.013732724823057652,
    -0.015976158902049065,
    0.00862953532487154,
    -0.0244789458

#### References

1. https://logz.io/blog/elasticsearch-mapping/#:~:text=Within%20a%20search%20engine%2C%20mapping,indexes%20and%20stores%20its%20fields

2. https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

3. https://www.elastic.co/search-labs/tutorials

4. https://www.elastic.co/search-labs/blog/text-similarity-search-with-vectors-in-elasticsearch